[View in Colaboratory](https://colab.research.google.com/github/mahedjaved/tutorial-rep/blob/master/1_GettingStarted_EagerExec.ipynb)

# Chapter 1 - Getting Start - Exec

* Currently many APIs in tensorflow
* EagerExec is the easiest high level API in TF

In [0]:
#!pip install -q --upgrade tensorflow
#!pip freeze
#!pip install mxnet

### 1.1 Basic Guid for Eager Execution

* Eager execution enables tensorflow to turn off **computational graph mode** and begin with imperative execution

---
Begin by importing libraries


In [0]:
from __future__ import absolute_import, division, print_function
import os
currDir=os.getcwd()
import matplotlib as plt

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()